In [25]:
import pandas as pd
import numpy as np
import read_player_stats
from sklearn.ensemble import RandomForestRegressor
from sklearn import cross_validation
from sklearn import metrics
pd.options.display.max_columns = 1000
import matplotlib.pyplot as plt
%matplotlib inline

In [26]:
pages = [0,1]
seasons = range(2004,2015)

total_df = None
for season in seasons:
    for page in pages:
        df = read_player_stats.rb_stats(season,page)
        if total_df is None:
            total_df = df
        else:
            total_df = total_df.append(df, ignore_index = True)
            
total_df.sort_index(by=['Name','Season'], inplace=True)
total_df.head()

,Name,Team,Games,RunAtt,RunYards,RunTD,Targets,Rec,RecYards,RecTD,FFP,FFPPG,Season
582,AaronBrown,DET,15,27,131,0,15,9,84,1,27.5,1.8,2009
59,AaronStecker,NO,16,58,244,2,38,29,174,0,53.8,3.4,2004
150,AaronStecker,NO,15,95,363,0,48,35,281,0,64.4,4.3,2005
286,AaronStecker,NO,12,4,11,0,28,19,190,0,20.1,1.7,2006
341,AaronStecker,NO,16,114,444,5,43,37,215,0,95.9,6.0,2007


In [27]:
def merge_seasons(df_season1, df_season2):
    df_dropped1 = df_season1.drop(['Season', 'Team'], axis=1)
    df_dropped2 = df_season2.drop(['Season', 'Team'], axis=1)
    
    merged = pd.merge(df_dropped1,df_dropped2, on='Name', how='outer', suffixes=('_1', '_2'))
    return merged

In [28]:
training_data_df = None
for season in range(2004,2014):
    df1 = total_df[total_df.Season == season]
    df2 = total_df[total_df.Season == season+1]
    label_df = total_df[total_df.Season == season+2]
    labeled = pd.merge(merge_seasons(df1, df2), label_df[['Name','FFP']], on='Name')
    if training_data_df is None:
        training_data_df = labeled
    else:
        training_data_df = training_data_df.append(labeled, ignore_index=True)
        
training_data_nadropped = training_data_df.dropna() #dont train on missing seasons

In [29]:
X_train = np.array(training_data_nadropped.drop(['Name','FFP'], axis=1))
y_train = np.array(training_data_nadropped['FFP'])

In [30]:
y_train

array([  20.1,  179.9,   38.7,  257.6,  186.2,   16. ,  111.3,  173.9,
        123.6,  185.5,  119.2,  173.6,  174.8,  178.7,  146.6,   51.4,
         75.1,   65.9,  168.9,   12.7,  418.3,   62.8,  189.9,  333.9,
         36.8,   27.9,   65. ,   65.9,   27.7,   71. ,   43.6,   53.4,
         21.9,   17.6,  116.7,   98.9,  215.2,   62.2,  136.4,   15.9,
        329.4,   26.8,  126. ,  172.3,  242.7,   16.8,  170.9,   17.3,
        161. ,  267.6,  150.6,   21.6,   95.9,  238.9,  111. ,   50.3,
        154.5,  282.4,   40.5,  104.1,  154.5,   89.2,  231.1,  129.8,
         43.2,  184.8,  189.8,  156. ,   34.4,  221.1,   90.7,   25.2,
         71.1,  125.8,  302.9,   97.6,   62.9,   98.5,  197.5,   26.3,
        114.1,   15.9,   45.8,   47.9,   39.2,   37.5,  110.3,   68.4,
        124.5,  129.1,   84.7,   49.6,   59.9,  109.2,  163.3,   56.9,
         30.5,  145.6,   99.6,   30.7,  119.6,  160. ,  191.8,   15.5,
        248.2,  248.2,   50.6,  202.5,  217.8,   51.6,  105.2,  115.8,
      

In [31]:
model = RandomForestRegressor(n_estimators=500)
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=500, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [32]:
training_data_nadropped

,Name,Games_1,RunAtt_1,RunYards_1,RunTD_1,Targets_1,Rec_1,RecYards_1,RecTD_1,FFP_1,FFPPG_1,Games_2,RunAtt_2,RunYards_2,RunTD_2,Targets_2,Rec_2,RecYards_2,RecTD_2,FFP_2,FFPPG_2,FFP
0,AaronStecker,16,58,244,2,38,29,174,0,53.8,3.4,15,95,363,0,48,35,281,0,64.4,4.3,20.1
1,AhmanGreen,15,259,1163,7,51,40,275,1,191.8,12.8,5,77,255,0,27,19,147,0,40.2,8.0,179.9
3,ArtosePinner,9,57,174,2,11,11,72,0,36.6,4.1,16,106,349,3,25,21,181,0,71.0,4.4,38.7
5,BrianWestbrook,13,177,812,3,87,73,703,6,205.5,15.8,12,157,614,3,96,61,616,4,165.0,13.8,257.6
6,ChesterTaylor,16,160,714,2,41,30,184,0,101.8,6.4,15,117,487,0,52,41,292,1,83.9,5.6,186.2
7,ChrisBrown,11,220,1067,6,33,20,147,0,157.4,14.3,15,224,851,5,36,25,327,2,159.8,10.7,16.0
8,ClintonPortis,15,343,1315,5,57,40,235,2,197.0,13.1,16,352,1516,11,41,30,216,0,239.2,15.0,111.3
9,CoreyDillon,15,345,1635,12,21,15,103,1,251.8,16.8,12,209,733,12,26,22,181,1,169.4,14.1,173.9
10,DeShaunFoster,4,59,255,2,9,9,76,0,45.1,11.3,15,206,882,2,48,34,372,1,143.4,9.6,123.6
11,DeuceMcAllister,14,269,1074,9,48,34,228,0,184.2,13.2,5,93,335,3,18,17,117,0,63.2,12.6,185.5
